Purpose of this notebook is to setup a simple VAE that can learn poses of cubes.

@yvan june 9 2018

In [1]:
import os
import torch
import torch.nn as nn

from torch.utils import data
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

In [ ]:
# constants
BATCH_SIZE = 64
SEED = 3030
IMG_PATH = '/home/yvan/data_load/imgs_jpg_clpr/'

In [4]:
torch.cuda.is_available(), torch.__version__, torch.cuda.device_count()

(True, '0.4.0', 2)